In [1]:
import pandas as pd
import joblib
import openai
from openai import AzureOpenAI


In [2]:

model = joblib.load('logistic_model.pkl')
label_encoders = joblib.load('label_encoders.pkl')

In [3]:
def preprocess_student_data(student_data, label_encoders):
    encoded_data = {}
    for feature, value in student_data.items():
        if feature in label_encoders:
            le = label_encoders[feature]
            if value in le.classes_:
                encoded_data[feature] = le.transform([value])[0]
            else:
                print(f"Warning: Unseen label '{value}' in feature '{feature}'. Skipping this feature.")
                return None
        else:
            encoded_data[feature] = value  # feature keeps the same
    return pd.DataFrame([encoded_data])


In [4]:
# generate prediction result
def generate_prediction(user_input, model, label_encoders):
    preprocessed_data = preprocess_student_data(user_input, label_encoders)
    if preprocessed_data is not None:
        prediction = model.predict(preprocessed_data)[0]
        return "above avg" if prediction == 1 else "below avg"
    else:
        return "Unable to generate prediction due to unseen labels."


In [6]:
# generate output
def generate_study_plan(user_input):
    learning_pace = generate_prediction(user_input, model, label_encoders)

    # MESSAGES = [
    #     {"role": "system", "content": "You are a helpful assistant."},
    #     {"role": "user", "content": f"My predicted learning pace is {learning_pace}. I want a personalized study plan to improve my learning efficiency."},
    #     {"role": "assistant", "content": "What subject would you like the study plan to focus on?"},
    #     {"role": "user", "content": "Mathematics, with a focus on calculus."},
    # ]
    MESSAGES = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"I need help creating a study plan. Based on my habits, my predicted learning pace is {learning_pace}. I want to improve my study efficiency."},
        {
            "role": "assistant",
            "content": "Sure, I'd be happy to help! What subjects or topics do you need to include in your study plan?",
        },
        {"role": "user", "content": "The plan should cover mathematics over the next month."},
        {
            "role": "assistant",
            "content": "Do you have any specific goals or exams for these subjects during this period?",
        },
        {"role": "user", "content": "Yes, I'm preparing for midterms and need to cover calculus in mathematics."}
    ]


    # azure openai
    client = AzureOpenAI(
        azure_endpoint="https://mango-bush-0a9e12903.5.azurestaticapps.net/api/v1",
        api_key="b7cf07cd-cb5d-4994-a95a-3d5dd5ae7c53",
        api_version="2024-02-01",
    )

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=MESSAGES
    )

    response_content = completion.choices[0].message.content
    return response_content


In [7]:
# example
user_input = {
    'sex': 'F', 'age': 17, 'address': 'U', 'famsize': 'GT3', 'Pstatus': 'T',
    'Medu': 2, 'Fedu': 2, 'Mjob': 'at_home', 'Fjob': 'teacher', 'reason': 'course',
    'guardian': 'mother', 'traveltime': 2, 'studytime': 2, 'failures': 0,
    'schoolsup': 'yes', 'famsup': 'yes', 'paid': 'no', 'activities': 'yes',
    'nursery': 'yes', 'higher': 'yes', 'internet': 'yes', 'romantic': 'no',
    'famrel': 4, 'freetime': 3, 'goout': 3, 'Dalc': 1, 'Walc': 1, 'health': 3,
    'absences': 4
}

In [8]:
study_plan = generate_study_plan(user_input)
print(study_plan)

Great, let's create a detailed study plan for your calculus midterms over the next month. To improve your study efficiency, we'll incorporate strategies such as active learning, regular review sessions, and breaks to keep you fresh.

### Week 1: Review and Fundamental Concepts
**Day 1-2: Limits and Continuity**
- Read textbook chapters on limits and continuity.
- Watch online tutorials or lectures.
- Practice basic problems.

**Day 3-4: Derivatives and Differentiation Rules**
- Review your notes on derivatives.
- Solve derivative problems including power, product, quotient, and chain rules.
- Use flashcards for differentiation formulas.

**Day 5: Application of Derivatives**
- Learn about critical points, optimization problems, and related rates.
- Practice problems involving real-life applications of derivatives.

**Day 6: Review of the Week's Topics**
- Summarize and rewrite key concepts.
- Redo practice problems you found challenging.
- Take a short practice quiz.

**Day 7: Rest and